<a href="https://colab.research.google.com/github/Miketheanalyst/sprout-price-tracker/blob/main/SproutPriceTracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gspread oauth2client

In [ ]:
CREDENTIALS_FILE = "/content/drive/My Drive/Sprout Price Traker/gothic-card-447415-h7-81ff247666ac.json"
SHEET_NAME = "/content/drive/My Drive/Sprout Price Traker/Sprout.Prices"


In [1]:
drive.mount('/content/drive')


NameError: name 'drive' is not defined

In [2]:
import requests
from bs4 import BeautifulSoup
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime
import smtplib
from email.mime.text import MIMEText
import gdown

# Google Sheet ID
SHEET_ID = "1rvlhtgdoQstmSnLqQH912iI-CJDrk25yUn8q4YljPeM"

# Product URLs
PRODUCT_URLS = [
    "https://alsuper.com/producto/mix-germinado-50",    # Column A
    "https://alsuper.com/producto/germinado-de-brocoli-121",  # Column B
    "https://alsuper.com/producto/alfalfa-1321",        # Column C
    "https://alsuper.com/producto/fenogreco-1322",      # Column D
    "https://alsuper.com/producto/trigo-1323",          # Column E
    "https://alsuper.com/producto/lenteja-1324"         # Column F
]

# Download JSON credentials from Google Drive
def download_credentials():
    url = "https://drive.google.com/uc?id=1Nexj-kJN-w0-lF9ZUkihhpGtyD8cfdrj"
    output = "/content/credentials.json"
    gdown.download(url, output, quiet=False)
    return output

# Authenticate and connect to Google Sheets
def authenticate_google_sheets():
    credentials_file = download_credentials()
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials_file, scope)
    client = gspread.authorize(credentials)
    return client

# Fetch product details
def fetch_product_details(url):
    try:
        response = requests.get(url, verify=False, timeout=10)  # Added verify=False for SSL warnings
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract product name
        name_element = soup.find("h1", class_="as-roboto-slab as-font-24 as-font-bold")
        product_name = name_element.text.strip() if name_element else "Unknown Product"

        # Extract price
        price_element = soup.find("mat-label", class_="as-roboto-slab as-font-24 as-font-bold ng-star-inserted")
        if price_element:
            price_text = price_element.text.strip()
            product_price = float(price_text.replace("$", "").replace(",", ""))
        else:
            product_price = None

        return product_name, product_price
    except Exception as e:
        print(f"Error fetching product details from {url}: {e}")
        return "Error", None

# Compare prices and send email notification
def compare_and_notify(sheet, new_data):
    try:
        rows = sheet.get_all_values()
        if len(rows) < 2:
            return  # Not enough data to compare

        last_row = rows[-1][0:6]  # Prices are in columns A-F
        price_changes = []

        for i, new_price in enumerate(new_data[0:6]):  # Only compare prices, exclude timestamp
            if last_row[i] and new_price != "N/A" and float(last_row[i]) != float(new_price):
                price_changes.append((sheet.row_values(1)[i], last_row[i], new_price))

        if price_changes:
            send_email(price_changes, no_changes=False)
        else:
            send_email([], no_changes=True)  # Notify no price changes
    except Exception as e:
        print(f"Error comparing data or sending email: {e}")

# Send email no
def send_email(price_changes, no_changes):
    sender_email = "germinadosgh@gmail.com"
    sender_password = "wvenqraanckmlpnc"  # App password
    recipient_email = "emontemayor@yahoo.com"

    if no_changes:
        message_body = "No hubo ningun cambio en el precio."
        subject = "No hay cambio de precio de Germinado"
    else:
        message_body = "Price changes detected:\n\n"
        for change in price_changes:
            message_body += f"{change[0]}: Old Price: {change[1]}, New Price: {change[2]}\n"
        subject = "Price Changes Detected"

    msg = MIMEText(message_body)
    msg["Subject"] = subject
    msg["From"] = sender_email
    msg["To"] = recipient_email

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, recipient_email, msg.as_string())
            print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")

# Update Google Sheet
def update_google_sheet(sheet, data):
    try:
        sheet.append_row(data)
    except Exception as e:
        print(f"Error updating Google Sheet: {e}")

if __name__ == "__main__":
    print("Starting script...")
    client = authenticate_google_sheets()
    try:
        sheet = client.open_by_key(SHEET_ID).sheet1
        print("Accessing Google Sheet: Success")
    except Exception as e:
        print(f"Error accessing Google Sheet: {e}")
        exit()

    # Fetch product details
    new_data = []
    for url in PRODUCT_URLS:
        product_name, product_price = fetch_product_details(url)
        if product_price is not None:
            print(f"Product: {product_name}, Price: ${product_price:.2f}")
            new_data.append(product_price)
        else:
            new_data.append("N/A")

    # Add timestamp in the last column (Column G)
    new_data.append(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

    # Compare and update
    compare_and_notify(sheet, new_data)
    update_google_sheet(sheet, new_data)
    print("Script completed.")

Starting script...


Downloading...
From: https://drive.google.com/uc?id=1Nexj-kJN-w0-lF9ZUkihhpGtyD8cfdrj
To: /content/credentials.json
100%|██████████| 2.40k/2.40k [00:00<00:00, 11.7MB/s]


Accessing Google Sheet: Success


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'alsuper.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Product: Unknown Product, Price: $39.90


...

Product: Unknown Product, Price: $39.90


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'alsuper.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Product: Unknown Product, Price: $39.90


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'alsuper.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Product: Unknown Product, Price: $39.90
Email sent successfully!
Script completed.
